In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

In [ ]:
root = 'https://www.nfl.com'
url1 = "https://www.nfl.com/stats/player-stats/category/"
url2 = "reg/all/"
year = "2023/"
cat = "receiving/"
caturl = "receivingyards/desc"

url = url1 + cat + year + url2 + caturl
print("fetching at url: " + url)
headers = []
data = []

fetching at url: https://www.nfl.com/stats/player-stats/category/receiving/2023/reg/all/receivingyards/desc


In [ ]:
response = requests.get(url)
webpage = response.content
soup = BeautifulSoup(webpage, "html.parser")

In [ ]:
table = soup.find('table' ,{'class:', 'd3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable'})
rows = table('tr')
for row in rows:
  for tag in row:
    if tag.name == "th":
      headers.append(tag.get_text().strip())
headers

['Player',
 'Rec',
 'Yds',
 'TD',
 '20+',
 '40+',
 'LNG',
 'Rec 1st',
 '1st%',
 'Rec FUM',
 'Rec YAC/R',
 'Tgts']

In [ ]:
link  = soup.find('a', { 'class' : 'nfl-o-table-pagination__next' })
print(link)

for row in rows:
  curr= []
  for tag in row:
    if tag.name == "td":
      curr.append(tag.get_text().strip())
  if curr:
    data.append(curr)


<a class="nfl-o-table-pagination__next" href="/stats/player-stats/category/receiving/2023/REG/all/receivingyards/DESC?aftercursor=AAAAGQAAABlAj8AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNREUySWl3aU16SXdNRFV5TkRrdE5EUTJOUzB5TXpnMExUY3lOVE10TTJFMFlqUTJOakl5WVdFNUlpd2lNakF5TXlKZGZRPT0=" title="Next Page">
        Next Page
      </a>


In [ ]:
while link is not None:
  next_url = root + link['href']
  response = requests.get(next_url)
  webpage = response.content
  soup = BeautifulSoup(webpage, "html.parser")
  print(next_url)
  time.sleep(1)
  link  = soup.find('a', { 'class' : 'nfl-o-table-pagination__next' })
  table = soup.find('table' ,{'class:', 'd3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable'})
  rows = table('tr')
  for row in rows:
      cur_row = []
      for tag in row:
          if tag.name == 'td':
              cur_row.append(tag.get_text().strip())
      if cur_row: # only add the row if it has data
            data.append(cur_row)

https://www.nfl.com/stats/player-stats/category/receiving/2023/REG/all/receivingyards/DESC?aftercursor=AAAAGQAAABlAj8AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNREUySWl3aU16SXdNRFV5TkRrdE5EUTJOUzB5TXpnMExUY3lOVE10TTJFMFlqUTJOakl5WVdFNUlpd2lNakF5TXlKZGZRPT0=
https://www.nfl.com/stats/player-stats/category/receiving/2023/REG/all/receivingyards/DESC?aftercursor=AAAAMgAAADJAhwgAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STNNemNpTENJek1qQXdOR0kxTlMwMU1EVXpMVFExT1RjdFlqazFPQzA0TnpRd09HTXpNVGsxTm1ZaUxDSXlNREl6SWwxOQ==
https://www.nfl.com/stats/player-stats/category/receiving/2023/REG/all/receivingyards/DESC?aftercursor=AAAASwAAAEtAgagAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFOalVpTENJek1qQXdOREkwTlMwME16VTNMVE15TlRJdE1qWTBZaTB4WkdFelpUa3paREkzTkRVaUxDSXlNREl6SWwxOQ==
https://www.nfl.com/stats/player-stats/category/receiving/2023/REG/all/receivingyards/DESC?aftercursor=AAAAZAAAAGRAe0AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBNellpTENJek1qQXdORFUwWWkwME5URTRMVGswTXpBdFlqaGlOaTB5TVRJNFlqVmlNMlV5T0RJaUxDSXlNR

In [ ]:
stats = pd.DataFrame(data, columns = headers)
stats = stats.drop(columns = ['20+', '40+','Rec 1st', '1st%'])
stats

,Player,Rec,Yds,TD,LNG,Rec FUM,Rec YAC/R,Tgts
0,Tyreek Hill,119,1799,13,78,1,652,171
1,CeeDee Lamb,135,1749,12,92,3,680,181
2,Amon-Ra St. Brown,119,1515,10,70,1,668,164
3,Puka Nacua,105,1486,6,80,1,639,160
4,A.J. Brown,106,1456,7,59,2,473,158
...,...,...,...,...,...,...,...,...
486,Geno Smith,1,-2,0,-2,0,6,1
487,Robert Hainsey,0,-3,0,-3,0,-3,0
488,Desmond Ridder,1,-6,0,-6,0,0,1
489,Jake Browning,1,-7,0,-7,0,0,1


In [ ]:
stats.to_csv('receiving2023.csv',index=False)